<a href="https://colab.research.google.com/github/velmer/retrieval-information/blob/master/lab-06/Modelo_Vetorial_ri_lab_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
import numpy as np
import nltk
import re
import collections
import bisect
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Carregando dados

In [48]:
data = pd.read_csv('https://raw.githubusercontent.com/velmer/retrieval-information/master/lab-06/results.csv')
data.head(10)

,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...
5,Rosângela uma das crianças sequestradas por f...,“Roubaram minha identidade” diz Rosângela Par...,Joana Oliveira,30/03/2019 22:21:08,Brasil,Rosângela Serra Paraná não sabe quantos anos t...,https://brasil.elpais.com/brasil/2019/03/29/po...
6,“Lógica de usar torturadores da ditadura no cr...,Aloy Jupiara coautor de 'Os porões da contrav...,Felipe Betim,01/04/2019 13:22:17,Brasil,Trajetória similar tiveram outros vários agent...,https://brasil.elpais.com/brasil/2019/03/29/po...
7,Bolsonaro troca embaixada por escritório em Je...,Autoridade Palestina convoca embaixador no Bra...,Juan Carlos Sanz,01/04/2019 12:26:32,Brasil,A devoção dos cristãos evangélicos em seu apoi...,https://brasil.elpais.com/brasil/2019/03/31/po...
8,O lado mais sombrio de Dickens,Escritor e jornalista tentou internar sua mulh...,Rafa de Miguel,10/03/2019 14:29:37,Cultura,O homem mais famoso da era vitoriana o “poeta...,https://brasil.elpais.com/brasil/2019/03/01/cu...
9,O amor entre mulheres que sacudiu o Paraguai,O drama ganhou três prêmios em Berlim e provoc...,Gregorio Belinchón,09/03/2019 12:10:23,Cultura,Depois de inclusive o da crítica internacion...,https://brasil.elpais.com/brasil/2019/03/07/cu...


# 1. Reconstruindo Índice Invertido

In [0]:
tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê\-]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese')
tokenData = pd.DataFrame(data['text'], columns=['text'])
tokenData['tokenized_text'] = data.apply(lambda row: tknz.tokenize(row['text'].lower()), axis=1)

In [0]:
def is_tokenized_word(word):
  """
    Returns if the specified word matches all requirements
    to be a valid token.
  """
  return len(word) > 1 and any(c.isalnum() for c in word)

def build_index():
  index = {}
  num_document = 0

  for text in tokenData.tokenized_text:
    num_document += 1
    words_set = set(text)
    for word in words_set:
      if is_tokenized_word(word) and word.lower() not in stopwords:
        if word not in index.keys():
          index[word] = { 'ranking': [] }
        index[word]['ranking'].append((num_document, text.count(word)))

  return index

In [51]:
index = build_index()
indexed_data = pd.DataFrame()
indexed_data['Word'] = [word for word in index.keys()]
indexed_data['Documents (With Count)'] = [word['ranking'] for word in index.values()]
indexed_data.head(10)

,Word,Documents (With Count)
0,ruptura,"[(1, 1), (9, 1), (25, 1), (65, 1), (83, 1), (1..."
1,prevalência,"[(1, 2), (123, 1)]"
2,preferem,"[(1, 1), (63, 1), (65, 1), (125, 1), (172, 1),..."
3,vítimas,"[(1, 1), (3, 1), (5, 1), (6, 1), (29, 3), (40,..."
4,ordem,"[(1, 3), (2, 1), (14, 1), (25, 5), (63, 1), (6..."
5,história,"[(1, 1), (3, 13), (5, 1), (6, 4), (8, 1), (9, ..."
6,atos,"[(1, 1), (9, 1), (29, 1), (31, 1), (88, 1), (9..."
7,ultradireita,"[(1, 1), (2, 1), (175, 2), (204, 1)]"
8,pontua,"[(1, 1), (101, 1)]"
9,emedebista,"[(1, 1), (206, 1)]"


# 2. Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário

In [0]:
M = data.text.count()
def add_idf_to_index():
  for word in index:
    k = len(index[word]['ranking'])
    IDF = round(np.log((M+1)/k),2)
    index[word]['IDF'] = IDF

In [0]:
add_idf_to_index()

In [54]:
indexed_data['IDF'] = [word['IDF'] for word in index.values()]
indexed_data.head(10)

,Word,Documents (With Count),IDF
0,ruptura,"[(1, 1), (9, 1), (25, 1), (65, 1), (83, 1), (1...",3.44
1,prevalência,"[(1, 2), (123, 1)]",4.83
2,preferem,"[(1, 1), (63, 1), (65, 1), (125, 1), (172, 1),...",3.44
3,vítimas,"[(1, 1), (3, 1), (5, 1), (6, 1), (29, 3), (40,...",2.02
4,ordem,"[(1, 3), (2, 1), (14, 1), (25, 5), (63, 1), (6...",2.34
5,história,"[(1, 1), (3, 13), (5, 1), (6, 4), (8, 1), (9, ...",1.07
6,atos,"[(1, 1), (9, 1), (29, 1), (31, 1), (88, 1), (9...",2.75
7,ultradireita,"[(1, 1), (2, 1), (175, 2), (204, 1)]",4.14
8,pontua,"[(1, 1), (101, 1)]",4.83
9,emedebista,"[(1, 1), (206, 1)]",4.83


# 3. Implementar as seguintes versões do modelo vetorial:

### 3.1. Representação binária

In [0]:
def binary_vsm(query, document):
  score = 0
  query_tokens = query.split()
  document_tokens = document.split()
  
  for token in query_tokens:
    score += (token in document_tokens)
    
  return score

### 3.2. TF

In [0]:
def tf_vsm(query, document):
  score = 0
  document_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    score += document_tokens.count(word)
  
  return score

### 3.3. TF-IDF

In [0]:
def tfidf_vsm(query, document):
  score = 0
  document_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    cwd = document_tokens.count(word)
    score += cwd * index[word]['IDF']
  
  return round(score,2)

### 3.4. BM25

In [0]:
def bm25_vsm(query, document, k):
  score = 0
  document_tokens = document.split()
  query_tokens = query.split()
  
  words = [word for word in query_tokens if word in document_tokens]
    
  for word in words:
    cwd = document_tokens.count(word)
    dfw = len(index[word])
    score += (((k+1) * cwd) / (cwd + k)) * np.log10(((M+1) / dfw))
  
  return round(score,2)

# 4. Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta

In [0]:
queries = ['champions', 'bolsonaro', 'financeiro']

In [0]:
def get_top5_documents(query):
  db = []
  dtf = []
  dtfidf = []
  dbm25 = []
  n = 0
  for document in data.text:
    document = document.lower()
    bisect.insort(db, (binary_vsm(query, document), n))
    bisect.insort(dtf, (tf_vsm(query, document), n))
    bisect.insort(dtfidf, (tfidf_vsm(query, document), n))
    bisect.insort(dbm25, (bm25_vsm(query, document, 20), n))
    n += 1
  
  db.reverse()
  dtf.reverse()
  dtfidf.reverse()
  dbm25.reverse()
  
  return db[:5], dtf[:5], dtfidf[:5], dbm25[:5]

In [0]:
top5_binary = [[],[],[]]
top5_tf = [[],[],[]]
top5_tfidf = [[],[],[]]
top5_bm25 = [[],[],[]]

top5_binary[0], top5_tf[0], top5_tfidf[0], top5_bm25[0] = get_top5_documents(queries[0])
top5_binary[1], top5_tf[1], top5_tfidf[1], top5_bm25[1] = get_top5_documents(queries[1])
top5_binary[2], top5_tf[2], top5_tfidf[2], top5_bm25[2] = get_top5_documents(queries[2])

In [187]:
query_df = pd.DataFrame()
query_df['Query'] = queries
query_df['Binary'] = top5_binary
query_df['TF'] = top5_tf
query_df['TF-IDF'] = top5_tfidf
query_df['BM25'] = top5_bm25
query_df

,Query,Binary,TF,TF-IDF,BM25
0,champions,"[(1, 73), (1, 70), (1, 56), (1, 54), (1, 53)]","[(3, 40), (2, 70), (2, 53), (1, 73), (1, 56)]","[(9.96, 40), (6.64, 70), (6.64, 53), (3.32, 73), (3.32, 56)]","[(5.74, 40), (4.0, 70), (4.0, 53), (2.1, 73), (2.1, 56)]"
1,bolsonaro,"[(1, 247), (1, 239), (1, 237), (1, 236), (1, 235)]","[(32, 150), (30, 206), (30, 165), (19, 18), (12, 41)]","[(42.24, 150), (39.6, 206), (39.6, 165), (25.08, 18), (15.84, 41)]","[(27.1, 150), (26.42, 206), (26.42, 165), (21.45, 18), (16.51, 41)]"
2,financeiro,"[(1, 217), (1, 206), (1, 205), (1, 142), (1, 128)]","[(2, 128), (2, 33), (1, 217), (1, 206), (1, 205)]","[(5.92, 128), (5.92, 33), (2.96, 217), (2.96, 206), (2.96, 205)]","[(4.0, 128), (4.0, 33), (2.1, 217), (2.1, 206), (2.1, 205)]"


# 5. Compare os resultados encontrados e responda

## 5.1 Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta

In [0]:
def get_top1_score(top5):
  return [top[0][0] for top in top5]

def get_top1_doc(top5):
  return [top[0][1] for top in top5]

def get_doc_title(docs):
  return [data.title[doc] for doc in docs]

In [0]:
score_top5_binary = get_top1_score(top5_binary)
score_top5_tf = get_top1_score(top5_tf)
score_top5_tfidf = get_top1_score(top5_tfidf)
score_top5_bm25 = get_top1_score(top5_bm25)

doc_top5_binary = get_top1_doc(top5_binary)
doc_top5_tf = get_top1_doc(top5_tf)
doc_top5_tfidf = get_top1_doc(top5_tfidf)
doc_top5_bm25 = get_top1_doc(top5_bm25)

titles_binary = get_doc_title(doc_top5_binary)
titles_tf = get_doc_title(doc_top5_tf)
titles_tfidf = get_doc_title(doc_top5_tfidf)
titles_bm25 = get_doc_title(doc_top5_bm25)

### Binary

In [191]:
binary_res = pd.DataFrame()
binary_res['Query'] = queries
binary_res['Document'] = doc_top5_binary
binary_res['Document Title'] = titles_binary
binary_res['Binary Score'] = score_top5_binary
binary_res

,Query,Document,Document Title,Binary Score
0,champions,73,Manchester United marca nos acréscimos graças a VAR e derruba o PSG nas oitavas da Champions,1
1,bolsonaro,247,Tatiana Roque: “O problema da esquerda não é a pauta dita identitária mas sim a lacração”,1
2,financeiro,217,Desconfiança dos investidores com Bolsonaro cresce e dólar supera 4 reais,1


### TF

In [192]:
tf_res = pd.DataFrame()
tf_res['Query'] = queries
tf_res['Document'] = doc_top5_tf
tf_res['Document Title'] = titles_tf
tf_res['TF Score'] = score_top5_tf
tf_res

,Query,Document,Document Title,TF Score
0,champions,40,Turim se apaixona por Cristiano Ronaldo,3
1,bolsonaro,150,Quem mandou matar Marielle? E por quê?,32
2,financeiro,128,Wolfgang Streeck: “É preciso quebrar o euro e logo”,2


### TF-IDF

In [193]:
tfidf_res = pd.DataFrame()
tfidf_res['Query'] = queries
tfidf_res['Document'] = doc_top5_tfidf
tfidf_res['Document Title'] = titles_tfidf
tfidf_res['TF-IDF Score'] = score_top5_tfidf
tfidf_res

,Query,Document,Document Title,TF-IDF Score
0,champions,40,Turim se apaixona por Cristiano Ronaldo,9.96
1,bolsonaro,150,Quem mandou matar Marielle? E por quê?,42.24
2,financeiro,128,Wolfgang Streeck: “É preciso quebrar o euro e logo”,5.92


### BM25

In [194]:
bm25_res = pd.DataFrame()
bm25_res['Query'] = queries
bm25_res['Document'] = doc_top5_bm25
bm25_res['Document Title'] = titles_bm25
bm25_res['BM25 Score'] = score_top5_bm25
bm25_res

,Query,Document,Document Title,BM25 Score
0,champions,40,Turim se apaixona por Cristiano Ronaldo,5.74
1,bolsonaro,150,Quem mandou matar Marielle? E por quê?,27.10
2,financeiro,128,Wolfgang Streeck: “É preciso quebrar o euro e logo”,4.00


## 5.2 Calcule e reporte o overlap par-a-par entre os resultados de cada modelo (usando o índice de Jaccard)

In [0]:
def jaccard_similarity(a, b):
    s1 = set(a)
    s2 = set(b)
    return len(s1.intersection(s2)) / float(len(s1.union(s2)))

In [196]:
all_docs = []
all_docs.append(doc_top5_binary)
all_docs.append(doc_top5_bm25)
all_docs.append(doc_top5_tf)
all_docs.append(doc_top5_tfidf)

matrix = [['Measure', 'Binary', 'TF', 'TF-IDF', 'BM25']]
titles = matrix
for i in range(len(all_docs)):
  line = []
  for j in range(len(all_docs)):
    jaccard = jaccard_similarity(all_docs[i], all_docs[j])
    line.append(round(jaccard,2))
  matrix.append(line)
for i in range(1, len(matrix)):
  matrix[i].insert(0, matrix[0][i])
  
pd.DataFrame(matrix)

,0,1,2,3,4
0,Measure,Binary,TF,TF-IDF,BM25
1,Binary,1,0,0,0
2,TF,0,1,1,1
3,TF-IDF,0,1,1,1
4,BM25,0,1,1,1
